In [1]:
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

import geopy

import matplotlib.pyplot as plt
import seaborn as sns

# Configure image format to be ‘svg’
%config InlineBackend.figure_format = 'svg'

# allow visuals to render within notebook
%matplotlib inline 

In [2]:
# Make some room to see stuff (i.e. drop display limits on Pandas rows & cols - be careful w/ big df's!)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# 1. Import MTA Data From SQLite Database

In [3]:
engine = create_engine('sqlite:///../_PROJECT/mta_data.db')

engine.table_names()

/var/folders/16/my2sdbrn4hvfl3_91fx0wj9c0000gn/T/ipykernel_22642/152434287.py:3: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['mta_data']

In [4]:
# Check for any NULL values

pd.read_sql('''
            SELECT *
            FROM mta_data
            WHERE unit IS NULL 
               OR scp IS NULL
               OR station IS NULL
               OR linename IS NULL
               OR date IS NULL
               OR time IS NULL
               OR desc IS NULL
               OR entries IS NULL
               OR exits IS NULL;
            ''', con=engine)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS


In [5]:
# Bring in just the data for May-June 2019 (order by ascending 'DATE'; don't bother importing 'EXITS' col)

# df_2019 = pd.read_sql('''
#                       SELECT (STATION || ' - ' || LINENAME || ' - ' || DIVISION) AS STATION_ID,
#                          STATION AS STATION_NAME, [C/A], UNIT, SCP,  DATE, TIME, DESC, ENTRIES
#                       FROM mta_data
#                       WHERE DATE = '04/30/2019' OR DATE LIKE '05%19' OR DATE LIKE '06%19'
#                       ORDER BY DATE ASC;
#                       ''', con=engine)

df_2019 = pd.read_sql('''
                      SELECT *
                      FROM mta_data
                      WHERE DATE = '04/30/2019' OR DATE LIKE '05%19' OR DATE LIKE '06%19'
                      ORDER BY DATE ASC;
                      ''', con=engine)

In [6]:
# Add a 'DATETIME' column (and drop 'DATE' & 'TIME' - can get either of these from 'DATETIME' if needed)

df_2019['DATETIME'] = pd.to_datetime(df_2019['DATE'] + ' ' + df_2019['TIME'])

# Make 'DATE' column a proper date format
df_2019['DATE'] = df_2019['DATETIME'].dt.date

df_2019.drop(columns=['TIME', 'EXITS'], inplace=True)

In [7]:
print(df_2019.shape)
display(df_2019.head())
display(df_2019.tail())

(3639340, 10)


,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,DESC,ENTRIES,DATETIME
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-04-30,REGULAR,7038242,2019-04-30 00:00:00
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-04-30,REGULAR,7038261,2019-04-30 04:00:00
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-04-30,REGULAR,7038308,2019-04-30 08:00:00
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-04-30,REGULAR,7038474,2019-04-30 12:00:00
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-04-30,REGULAR,7038775,2019-04-30 16:00:00


,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,DESC,ENTRIES,DATETIME
3639335,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,2019-06-30,REGULAR,5554,2019-06-30 05:00:00
3639336,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,2019-06-30,REGULAR,5554,2019-06-30 09:00:00
3639337,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,2019-06-30,REGULAR,5554,2019-06-30 13:00:00
3639338,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,2019-06-30,REGULAR,5554,2019-06-30 17:00:00
3639339,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,2019-06-30,REGULAR,5554,2019-06-30 21:00:00


In [8]:
# Bring in just the data for May-June 2021 (order by ascending 'DATE'; don't bother importing 'EXITS' col)

# df_2021 = pd.read_sql('''
#                       SELECT (STATION || ' - ' || LINENAME || ' - ' || DIVISION) AS STATION_ID,
#                          STATION AS STATION_NAME, [C/A], UNIT, SCP,  DATE, TIME, DESC, ENTRIES
#                       FROM mta_data
#                       WHERE DATE = '04/30/2021' OR DATE LIKE '05%21' OR DATE LIKE '06%21'
#                       ORDER BY DATE ASC;
#                       ''', con=engine)

df_2021 = pd.read_sql('''
                      SELECT *
                      FROM mta_data
                      WHERE DATE = '04/30/2021' OR DATE LIKE '05%21' OR DATE LIKE '06%21'
                      ORDER BY DATE ASC;
                      ''', con=engine)

In [9]:
# Add a 'DATETIME' column (and drop 'DATE' & 'TIME' - can get either of these from 'DATETIME' if needed)

df_2021['DATETIME'] = pd.to_datetime(df_2021['DATE'] + ' ' + df_2021['TIME'])

# Make 'DATE' column a proper date format
df_2021['DATE'] = df_2021['DATETIME'].dt.date

df_2021.drop(columns=['TIME', 'EXITS'], inplace=True)

In [10]:
print(df_2021.shape)
display(df_2021.head())
display(df_2021.tail())

(3706606, 10)


,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,DESC,ENTRIES,DATETIME
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,2021-04-30,REGULAR,7564048,2021-04-30 00:00:00
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,2021-04-30,REGULAR,7564059,2021-04-30 04:00:00
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,2021-04-30,REGULAR,7564073,2021-04-30 08:00:00
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,2021-04-30,REGULAR,7564118,2021-04-30 12:00:00
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,2021-04-30,REGULAR,7564286,2021-04-30 16:00:00


,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,DESC,ENTRIES,DATETIME
3706601,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,2021-06-30,REGULAR,5554,2021-06-30 05:00:00
3706602,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,2021-06-30,REGULAR,5554,2021-06-30 09:00:00
3706603,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,2021-06-30,REGULAR,5554,2021-06-30 13:00:00
3706604,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,2021-06-30,REGULAR,5554,2021-06-30 17:00:00
3706605,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,2021-06-30,REGULAR,5554,2021-06-30 21:00:00


# 2. Basic Cleaning

### 2a) Check for Duplicates
#### A single turnstile somwhere in the system is uniquely identified by C/A (Booth) + UNIT (Station) + SCP (Turnstile)
#### Add a new column 'ID' that creates a unique identifier for grouping on these three

In [11]:
df_2019['ID'] = df_2019['C/A'] + ' - ' + df_2019['UNIT'] + ' - ' + df_2019['SCP']
df_2019.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,DESC,ENTRIES,DATETIME,ID
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-04-30,REGULAR,7038242,2019-04-30 00:00:00,A002 - R051 - 02-00-00
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-04-30,REGULAR,7038261,2019-04-30 04:00:00,A002 - R051 - 02-00-00
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-04-30,REGULAR,7038308,2019-04-30 08:00:00,A002 - R051 - 02-00-00
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-04-30,REGULAR,7038474,2019-04-30 12:00:00,A002 - R051 - 02-00-00
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-04-30,REGULAR,7038775,2019-04-30 16:00:00,A002 - R051 - 02-00-00


In [12]:
df_2021['ID'] = df_2021['C/A'] + ' - ' + df_2021['UNIT'] + ' - ' + df_2021['SCP']
df_2021.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,DESC,ENTRIES,DATETIME,ID
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,2021-04-30,REGULAR,7564048,2021-04-30 00:00:00,A002 - R051 - 02-00-00
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,2021-04-30,REGULAR,7564059,2021-04-30 04:00:00,A002 - R051 - 02-00-00
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,2021-04-30,REGULAR,7564073,2021-04-30 08:00:00,A002 - R051 - 02-00-00
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,2021-04-30,REGULAR,7564118,2021-04-30 12:00:00,A002 - R051 - 02-00-00
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,2021-04-30,REGULAR,7564286,2021-04-30 16:00:00,A002 - R051 - 02-00-00


#### Now check for duplicates (Start with df_2019):

In [13]:
check = df_2019.groupby(['ID', 'DATETIME'])['ENTRIES'] \
                        .count().sort_values(ascending=False)
check.reset_index().head()

,ID,DATETIME,ENTRIES
0,R249 - R179 - 01-05-00,2019-06-27 00:00:00,4
1,N045 - R187 - 01-06-00,2019-06-02 17:00:00,4
2,R174 - R034 - 00-00-03,2019-06-08 00:00:00,4
3,N045 - R187 - 01-00-00,2019-06-02 17:00:00,4
4,R174 - R034 - 00-00-02,2019-06-08 00:00:00,4


In [14]:
for i in range(0, 6):
    display(df_2019[(df_2019['ID'] == check.reset_index().iloc[i, 0:1].values[0]) & 
                    (df_2019['DATETIME'] == check.reset_index().iloc[i, 1:2].values[0])])

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,DESC,ENTRIES,DATETIME,ID
3427686,R249,R179,01-05-00,86 ST,456,IRT,2019-06-27,REGULAR,6,2019-06-27,R249 - R179 - 01-05-00
3427687,R249,R179,01-05-00,86 ST,456,IRT,2019-06-27,RECOVR AUD,11,2019-06-27,R249 - R179 - 01-05-00
3457150,R249,R179,01-05-00,86 ST,456,IRT,2019-06-27,REGULAR,6,2019-06-27,R249 - R179 - 01-05-00
3457151,R249,R179,01-05-00,86 ST,456,IRT,2019-06-27,RECOVR AUD,11,2019-06-27,R249 - R179 - 01-05-00


,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,DESC,ENTRIES,DATETIME,ID
1940800,N045,R187,01-06-00,81 ST-MUSEUM,BC,IND,2019-06-02,REGULAR,2254050,2019-06-02 17:00:00,N045 - R187 - 01-06-00
1940801,N045,R187,01-06-00,81 ST-MUSEUM,BC,IND,2019-06-02,RECOVR AUD,2254049,2019-06-02 17:00:00,N045 - R187 - 01-06-00
1969768,N045,R187,01-06-00,81 ST-MUSEUM,BC,IND,2019-06-02,REGULAR,2254050,2019-06-02 17:00:00,N045 - R187 - 01-06-00
1969769,N045,R187,01-06-00,81 ST-MUSEUM,BC,IND,2019-06-02,RECOVR AUD,2254049,2019-06-02 17:00:00,N045 - R187 - 01-06-00


,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,DESC,ENTRIES,DATETIME,ID
2306436,R174,R034,00-00-03,125 ST,1,IRT,2019-06-08,REGULAR,3828801,2019-06-08,R174 - R034 - 00-00-03
2306437,R174,R034,00-00-03,125 ST,1,IRT,2019-06-08,RECOVR AUD,3828804,2019-06-08,R174 - R034 - 00-00-03
2335466,R174,R034,00-00-03,125 ST,1,IRT,2019-06-08,REGULAR,3828801,2019-06-08,R174 - R034 - 00-00-03
2335467,R174,R034,00-00-03,125 ST,1,IRT,2019-06-08,RECOVR AUD,3828804,2019-06-08,R174 - R034 - 00-00-03


,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,DESC,ENTRIES,DATETIME,ID
1940782,N045,R187,01-00-00,81 ST-MUSEUM,BC,IND,2019-06-02,REGULAR,5254570,2019-06-02 17:00:00,N045 - R187 - 01-00-00
1940783,N045,R187,01-00-00,81 ST-MUSEUM,BC,IND,2019-06-02,RECOVR AUD,5254568,2019-06-02 17:00:00,N045 - R187 - 01-00-00
1969750,N045,R187,01-00-00,81 ST-MUSEUM,BC,IND,2019-06-02,REGULAR,5254570,2019-06-02 17:00:00,N045 - R187 - 01-00-00
1969751,N045,R187,01-00-00,81 ST-MUSEUM,BC,IND,2019-06-02,RECOVR AUD,5254568,2019-06-02 17:00:00,N045 - R187 - 01-00-00


,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,DESC,ENTRIES,DATETIME,ID
2306430,R174,R034,00-00-02,125 ST,1,IRT,2019-06-08,REGULAR,6091547,2019-06-08,R174 - R034 - 00-00-02
2306431,R174,R034,00-00-02,125 ST,1,IRT,2019-06-08,RECOVR AUD,6091548,2019-06-08,R174 - R034 - 00-00-02
2335460,R174,R034,00-00-02,125 ST,1,IRT,2019-06-08,REGULAR,6091547,2019-06-08,R174 - R034 - 00-00-02
2335461,R174,R034,00-00-02,125 ST,1,IRT,2019-06-08,RECOVR AUD,6091548,2019-06-08,R174 - R034 - 00-00-02


,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,DESC,ENTRIES,DATETIME,ID
2467844,N045,R187,01-06-00,81 ST-MUSEUM,BC,IND,2019-06-11,REGULAR,2258033,2019-06-11 21:00:00,N045 - R187 - 01-06-00
2467845,N045,R187,01-06-00,81 ST-MUSEUM,BC,IND,2019-06-11,RECOVR AUD,2258031,2019-06-11 21:00:00,N045 - R187 - 01-06-00
2496957,N045,R187,01-06-00,81 ST-MUSEUM,BC,IND,2019-06-11,REGULAR,2258033,2019-06-11 21:00:00,N045 - R187 - 01-06-00
2496958,N045,R187,01-06-00,81 ST-MUSEUM,BC,IND,2019-06-11,RECOVR AUD,2258031,2019-06-11 21:00:00,N045 - R187 - 01-06-00


#### Looks like accidental data duplication and some RECOV AUDs virtually identical to REGULARs; will just drop all the duplicates

In [15]:
print(df_2019.shape)

df_2019 = df_2019.sort_values(by=['ID', 'DATETIME'], 
                              ascending=False) \
                             .drop_duplicates(subset=['ID', 'DATETIME'])

df_2019.reset_index(drop=True, inplace=True)
print(df_2019.shape)

(3639340, 11)
(1819646, 11)


In [16]:
# Re-check for duplicates

check = df_2019.groupby(['ID', 'DATETIME'])['ENTRIES'] \
                        .count().sort_values(ascending=False)
check.reset_index().head()

,ID,DATETIME,ENTRIES
0,A002 - R051 - 02-00-00,2019-04-30 00:00:00,1
1,R142 - R293 - 01-00-01,2019-05-12 18:00:00,1
2,R142 - R293 - 01-00-01,2019-05-14 06:00:00,1
3,R142 - R293 - 01-00-01,2019-05-14 02:00:00,1
4,R142 - R293 - 01-00-01,2019-05-13 22:00:00,1


#### Now deal with duplicates in df_2021:

In [17]:
check = df_2021.groupby(['ID', 'DATETIME'])['ENTRIES'] \
                         .count().sort_values(ascending=False)
check.reset_index().head()

,ID,DATETIME,ENTRIES
0,B024 - R211 - 00-05-00,2021-05-02 12:00:00,4
1,B024 - R211 - 00-05-00,2021-05-05 12:00:00,4
2,N329 - R201 - 00-03-02,2021-05-08 05:00:00,4
3,N329 - R201 - 00-03-02,2021-05-08 01:00:00,4
4,N335 - R158 - 01-00-03,2021-06-20 18:30:00,4


In [18]:
for i in range(0, 6):
    display(df_2021[(df_2021['ID'] == check.reset_index().iloc[i, 0:1].values[0]) & 
                    (df_2021['DATETIME'] == check.reset_index().iloc[i, 1:2].values[0])])

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,DESC,ENTRIES,DATETIME,ID
121785,B024,R211,00-05-00,KINGS HWY,BQ,BMT,2021-05-02,REGULAR,262152,2021-05-02 12:00:00,B024 - R211 - 00-05-00
121786,B024,R211,00-05-00,KINGS HWY,BQ,BMT,2021-05-02,RECOVR AUD,13,2021-05-02 12:00:00,B024 - R211 - 00-05-00
151574,B024,R211,00-05-00,KINGS HWY,BQ,BMT,2021-05-02,REGULAR,262152,2021-05-02 12:00:00,B024 - R211 - 00-05-00
151575,B024,R211,00-05-00,KINGS HWY,BQ,BMT,2021-05-02,RECOVR AUD,13,2021-05-02 12:00:00,B024 - R211 - 00-05-00


,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,DESC,ENTRIES,DATETIME,ID
301051,B024,R211,00-05-00,KINGS HWY,BQ,BMT,2021-05-05,REGULAR,262152,2021-05-05 12:00:00,B024 - R211 - 00-05-00
301052,B024,R211,00-05-00,KINGS HWY,BQ,BMT,2021-05-05,RECOVR AUD,21,2021-05-05 12:00:00,B024 - R211 - 00-05-00
330870,B024,R211,00-05-00,KINGS HWY,BQ,BMT,2021-05-05,REGULAR,262152,2021-05-05 12:00:00,B024 - R211 - 00-05-00
330871,B024,R211,00-05-00,KINGS HWY,BQ,BMT,2021-05-05,RECOVR AUD,21,2021-05-05 12:00:00,B024 - R211 - 00-05-00


,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,DESC,ENTRIES,DATETIME,ID
490387,N329,R201,00-03-02,WOODHAVEN BLVD,MR,IND,2021-05-08,REGULAR,4825280,2021-05-08 05:00:00,N329 - R201 - 00-03-02
490388,N329,R201,00-03-02,WOODHAVEN BLVD,MR,IND,2021-05-08,RECOVR AUD,12538860,2021-05-08 05:00:00,N329 - R201 - 00-03-02
520213,N329,R201,00-03-02,WOODHAVEN BLVD,MR,IND,2021-05-08,REGULAR,4825280,2021-05-08 05:00:00,N329 - R201 - 00-03-02
520214,N329,R201,00-03-02,WOODHAVEN BLVD,MR,IND,2021-05-08,RECOVR AUD,12538860,2021-05-08 05:00:00,N329 - R201 - 00-03-02


,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,DESC,ENTRIES,DATETIME,ID
490385,N329,R201,00-03-02,WOODHAVEN BLVD,MR,IND,2021-05-08,REGULAR,4825276,2021-05-08 01:00:00,N329 - R201 - 00-03-02
490386,N329,R201,00-03-02,WOODHAVEN BLVD,MR,IND,2021-05-08,RECOVR AUD,12538859,2021-05-08 01:00:00,N329 - R201 - 00-03-02
520211,N329,R201,00-03-02,WOODHAVEN BLVD,MR,IND,2021-05-08,REGULAR,4825276,2021-05-08 01:00:00,N329 - R201 - 00-03-02
520212,N329,R201,00-03-02,WOODHAVEN BLVD,MR,IND,2021-05-08,RECOVR AUD,12538859,2021-05-08 01:00:00,N329 - R201 - 00-03-02


,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,DESC,ENTRIES,DATETIME,ID
3061698,N335,R158,01-00-03,KEW GARDENS,EF,IND,2021-06-20,REGULAR,5201958,2021-06-20 18:30:00,N335 - R158 - 01-00-03
3061699,N335,R158,01-00-03,KEW GARDENS,EF,IND,2021-06-20,RECOVR AUD,5201960,2021-06-20 18:30:00,N335 - R158 - 01-00-03
3091640,N335,R158,01-00-03,KEW GARDENS,EF,IND,2021-06-20,REGULAR,5201958,2021-06-20 18:30:00,N335 - R158 - 01-00-03
3091641,N335,R158,01-00-03,KEW GARDENS,EF,IND,2021-06-20,RECOVR AUD,5201960,2021-06-20 18:30:00,N335 - R158 - 01-00-03


,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,DESC,ENTRIES,DATETIME,ID
181380,B024,R211,00-05-00,KINGS HWY,BQ,BMT,2021-05-03,REGULAR,262152,2021-05-03 20:00:00,B024 - R211 - 00-05-00
181381,B024,R211,00-05-00,KINGS HWY,BQ,BMT,2021-05-03,RECOVR AUD,16,2021-05-03 20:00:00,B024 - R211 - 00-05-00
211194,B024,R211,00-05-00,KINGS HWY,BQ,BMT,2021-05-03,REGULAR,262152,2021-05-03 20:00:00,B024 - R211 - 00-05-00
211195,B024,R211,00-05-00,KINGS HWY,BQ,BMT,2021-05-03,RECOVR AUD,16,2021-05-03 20:00:00,B024 - R211 - 00-05-00


#### Looks like a similar situation as with df_2019; some RECOVR AUDs look like trivial tests of some sort; will just drop the duplicates

In [19]:
print(df_2021.shape)

df_2021 = df_2021.sort_values(by=['ID', 'DATETIME'], 
                              ascending=False) \
                             .drop_duplicates(subset=['ID', 'DATETIME'])

df_2021.reset_index(drop=True, inplace=True)
print(df_2021.shape)

(3706606, 11)
(1853257, 11)


In [20]:
# Re-check for duplicates

check = df_2021.groupby(['ID', 'DATETIME'])['ENTRIES'] \
                         .count().sort_values(ascending=False)
check.reset_index().head()

,ID,DATETIME,ENTRIES
0,A002 - R051 - 02-00-00,2021-04-30 00:00:00,1
1,R138 - R293 - 00-03-04,2021-05-08 22:00:00,1
2,R138 - R293 - 00-03-04,2021-05-13 06:00:00,1
3,R138 - R293 - 00-03-04,2021-05-13 02:00:00,1
4,R138 - R293 - 00-03-04,2021-05-12 22:00:00,1


### 2b) Find common set of Stations in 2019 and 2021 datasets (Station roughly equates to the 'UNIT' column)

In [21]:
print(df_2019['UNIT'].unique().shape)
print(df_2021['UNIT'].unique().shape)

print(np.setdiff1d(df_2019['UNIT'].unique(), df_2021['UNIT'].unique()))
print(np.setdiff1d(df_2021['UNIT'].unique(), df_2019['UNIT'].unique()))

(469,)
(469,)
['R260']
['R094']


#### Each year's data has one station not present in the other - will find the common set between the two years

In [22]:
common_units = np.intersect1d(df_2019['UNIT'].unique(), df_2021['UNIT'].unique())
common_units.shape

(468,)

#### Now limit both year's datasets down to these common stations (UNITs)

In [23]:
print(df_2019.shape)
df_2019 = df_2019[df_2019['UNIT'].isin(common_units)]
df_2019.reset_index(drop=True, inplace=True)
print(df_2019.shape)

(1819646, 11)
(1817754, 11)


In [24]:
print(df_2021.shape)
df_2021 = df_2021[df_2021['UNIT'].isin(common_units)]
df_2021.reset_index(drop=True, inplace=True)
print(df_2021.shape)

(1853257, 11)
(1848056, 11)


### 2c) Find NET_DAILY_ENTRIES per unique turnstile ('ID')

#### Assumption 1: Some turnstiles count backwards, accounting for the negatives; just take absolute value of everything
#### Assumption 2: Even considering rush hr., >180 entries/hr ON AVERAGE over the full day (24 hours) is unlikely, so daily entries count > 24*180 = 4320 is probably due to counter rollover or other asynchronous counter reset -> just set these values to 0

In [25]:
daily_per_turnstile_2019 = df_2019.groupby(['ID', 'DATE']).first().reset_index().drop(columns='ENTRIES')

daily_per_turnstile_2019['NET_DAILY_ENTRIES'] = np.abs(df_2019.groupby(['ID', 'DATE'])['ENTRIES'].first() - 
                                                       df_2019.groupby(['ID', 'DATE'])['ENTRIES'].last()) \
                                                        .values

daily_per_turnstile_2019['NET_DAILY_ENTRIES'] = daily_per_turnstile_2019['NET_DAILY_ENTRIES'] \
                                                        .apply(lambda x: 0 if x > 180*24 else x)

daily_per_turnstile_2019.head()

,ID,DATE,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DESC,DATETIME,NET_DAILY_ENTRIES
0,A002 - R051 - 02-00-00,2019-04-30,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,2019-04-30 20:00:00,1278
1,A002 - R051 - 02-00-00,2019-05-01,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,2019-05-01 20:00:00,1346
2,A002 - R051 - 02-00-00,2019-05-02,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,2019-05-02 20:00:00,1327
3,A002 - R051 - 02-00-00,2019-05-03,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,2019-05-03 20:00:00,1326
4,A002 - R051 - 02-00-00,2019-05-04,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,2019-05-04 20:00:00,560


In [26]:
# This looks reasonable:

daily_per_turnstile_2019['NET_DAILY_ENTRIES'].describe()

count    300165.000000
mean        915.348638
std         851.082939
min           0.000000
25%         222.000000
50%         704.000000
75%        1394.000000
max        4320.000000
Name: NET_DAILY_ENTRIES, dtype: float64

#### Repeat this whole process for df_2021

In [27]:
daily_per_turnstile_2021 = df_2021.groupby(['ID', 'DATE']).first().reset_index().drop(columns='ENTRIES')

daily_per_turnstile_2021['NET_DAILY_ENTRIES'] = np.abs(df_2021.groupby(['ID', 'DATE'])['ENTRIES'].first() - 
                                                       df_2021.groupby(['ID', 'DATE'])['ENTRIES'].last()) \
                                                        .values

daily_per_turnstile_2021['NET_DAILY_ENTRIES'] = daily_per_turnstile_2021['NET_DAILY_ENTRIES'] \
                                                        .apply(lambda x: 0 if x > 180*24 else x)

daily_per_turnstile_2021.head()

,ID,DATE,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DESC,DATETIME,NET_DAILY_ENTRIES
0,A002 - R051 - 02-00-00,2021-04-30,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,2021-04-30 20:00:00,469
1,A002 - R051 - 02-00-00,2021-05-01,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,2021-05-01 20:00:00,361
2,A002 - R051 - 02-00-00,2021-05-02,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,2021-05-02 20:00:00,250
3,A002 - R051 - 02-00-00,2021-05-03,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,2021-05-03 20:00:00,472
4,A002 - R051 - 02-00-00,2021-05-04,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,2021-05-04 20:00:00,501


In [28]:
# This looks reasonable:

daily_per_turnstile_2021['NET_DAILY_ENTRIES'].describe()

count    308646.000000
mean        320.566260
std         327.177385
min           0.000000
25%          73.000000
50%         233.000000
75%         466.000000
max        3808.000000
Name: NET_DAILY_ENTRIES, dtype: float64

#### While we're here, let's add a WEEKDAY column (1 for M-F, 0 for Sat or Sun)

In [29]:
# pd.Series.dt.weekday: 0 = Mon, 6 = Sun

daily_per_turnstile_2019['WEEKDAY'] = pd.to_datetime(daily_per_turnstile_2019['DATE']) \
                                                        .dt.weekday.apply(lambda x: 1 if x < 5 else 0)
display(daily_per_turnstile_2019.sample(10))

,ID,DATE,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DESC,DATETIME,NET_DAILY_ENTRIES,WEEKDAY
288295,R618 - R058 - 01-00-02,2019-05-28,R618,R058,01-00-02,BERGEN ST,23,IRT,REGULAR,2019-05-28 20:00:00,308,1
286474,R610 - R057 - 00-05-00,2019-05-04,R610,R057,00-05-00,ATL AV-BARCLAY,2345BDNQR,IRT,REGULAR,2019-05-04 20:00:00,1,0
257451,R334 - R367 - 00-00-01,2019-06-09,R334,R367,00-00-01,233 ST,25,IRT,REGULAR,2019-06-09 21:00:00,766,0
285461,R609 - R056 - 01-00-02,2019-06-12,R609,R056,01-00-02,NEVINS ST,2345,IRT,REGULAR,2019-06-12 20:00:00,1755,1
111627,N303 - R015 - 00-00-00,2019-06-16,N303,R015,00-00-00,5 AV/53 ST,EM,IND,REGULAR,2019-06-16 22:00:00,524,0
186499,PTH22 - R540 - 00-05-02,2019-05-19,PTH22,R540,00-05-02,PATH NEW WTC,1,PTH,REGULAR,2019-05-19 23:03:38,0,0
24166,B022 - R229 - 00-00-03,2019-05-14,B022,R229,00-00-03,AVENUE M,BQ,BMT,REGULAR,2019-05-14 20:00:00,1092,1
213691,R176 - R169 - 00-00-03,2019-06-12,R176,R169,00-00-03,137 ST CITY COL,1,IRT,REGULAR,2019-06-12 21:00:00,2578,1
31377,C012 - R258 - 01-03-02,2019-06-07,C012,R258,01-03-02,4AV-9 ST,DFGMNR,BMT,REGULAR,2019-06-07 20:00:00,877,1
219383,R202 - R042 - 00-00-03,2019-06-12,R202,R042,00-00-03,BOWLING GREEN,45,IRT,REGULAR,2019-06-12 20:00:00,2146,1


In [30]:
daily_per_turnstile_2021['WEEKDAY'] = pd.to_datetime(daily_per_turnstile_2021['DATE']) \
                                                        .dt.weekday.apply(lambda x: 1 if x < 5 else 0)
display(daily_per_turnstile_2021.sample(10))

,ID,DATE,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DESC,DATETIME,NET_DAILY_ENTRIES,WEEKDAY
30172,C008 - R099 - 00-00-01,2021-05-06,C008,R099,00-00-01,DEKALB AV,BDNQR,BMT,REGULAR,2021-05-06 20:00:00,374,1
25793,B023 - R211 - 01-06-01,2021-05-22,B023,R211,01-06-01,KINGS HWY,BQ,BMT,REGULAR,2021-05-22 20:00:00,358,0
171966,N702A - R572 - 01-00-00,2021-05-02,N702A,R572,01-00-00,96 ST-2 AVE,Q,IND,REGULAR,2021-05-02 21:00:00,252,0
202060,R124 - R290 - 03-00-01,2021-05-20,R124,R290,03-00-01,HOUSTON ST,1,IRT,REGULAR,2021-05-20 21:00:00,112,1
296838,R621 - R060 - 00-00-01,2021-05-30,R621,R060,00-00-01,EASTN PKWY-MUSM,23,IRT,REGULAR,2021-05-30 20:00:00,159,0
152047,N513 - R163 - 04-00-02,2021-05-14,N513,R163,04-00-02,14 ST,FLM123,IND,REGULAR,2021-05-14 20:00:00,181,1
258554,R306 - R207 - 00-00-04,2021-06-20,R306,R207,00-00-04,135 ST,23,IRT,REGULAR,2021-06-20 20:00:00,535,0
201915,R123 - R290 - 00-00-01,2021-06-30,R123,R290,00-00-01,HOUSTON ST,1,IRT,REGULAR,2021-06-30 21:00:00,389,1
65123,K022 - R402 - 00-03-01,2021-06-11,K022,R402,00-03-01,SENECA AVE,M,BMT,REGULAR,2021-06-11 21:00:00,431,1
35477,C023 - R213 - 00-00-01,2021-06-10,C023,R213,00-00-01,BAY RIDGE AV,R,BMT,REGULAR,2021-06-10 20:00:00,303,1


#### Drop the April 30th Data

In [31]:
print(daily_per_turnstile_2019.shape)
daily_per_turnstile_2019 = daily_per_turnstile_2019[ \
                                pd.to_datetime(daily_per_turnstile_2019['DATE']) != pd.to_datetime('2019-04-30')]
daily_per_turnstile_2019.reset_index(drop=True, inplace=True)
print(daily_per_turnstile_2019.shape)
display(daily_per_turnstile_2019.head())
display(daily_per_turnstile_2019.tail())

(300165, 12)
(295338, 12)


,ID,DATE,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DESC,DATETIME,NET_DAILY_ENTRIES,WEEKDAY
0,A002 - R051 - 02-00-00,2019-05-01,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,2019-05-01 20:00:00,1346,1
1,A002 - R051 - 02-00-00,2019-05-02,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,2019-05-02 20:00:00,1327,1
2,A002 - R051 - 02-00-00,2019-05-03,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,2019-05-03 20:00:00,1326,1
3,A002 - R051 - 02-00-00,2019-05-04,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,2019-05-04 20:00:00,560,0
4,A002 - R051 - 02-00-00,2019-05-05,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,2019-05-05 20:00:00,449,0


,ID,DATE,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DESC,DATETIME,NET_DAILY_ENTRIES,WEEKDAY
295333,TRAM2 - R469 - 00-05-01,2019-06-26,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,REGULAR,2019-06-26 21:00:00,0,1
295334,TRAM2 - R469 - 00-05-01,2019-06-27,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,REGULAR,2019-06-27 21:00:00,0,1
295335,TRAM2 - R469 - 00-05-01,2019-06-28,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,REGULAR,2019-06-28 21:00:00,0,1
295336,TRAM2 - R469 - 00-05-01,2019-06-29,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,REGULAR,2019-06-29 21:00:00,0,0
295337,TRAM2 - R469 - 00-05-01,2019-06-30,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,REGULAR,2019-06-30 21:00:00,0,0


In [32]:
# Drop the 4/30/2021 rows

print(daily_per_turnstile_2021.shape)
daily_per_turnstile_2021 = daily_per_turnstile_2021[ \
                                pd.to_datetime(daily_per_turnstile_2021['DATE']) != pd.to_datetime('2021-04-30')]
daily_per_turnstile_2019.reset_index(drop=True, inplace=True)
print(daily_per_turnstile_2021.shape)
display(daily_per_turnstile_2021.head())
display(daily_per_turnstile_2021.tail())

(308646, 12)
(303670, 12)


,ID,DATE,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DESC,DATETIME,NET_DAILY_ENTRIES,WEEKDAY
1,A002 - R051 - 02-00-00,2021-05-01,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,2021-05-01 20:00:00,361,0
2,A002 - R051 - 02-00-00,2021-05-02,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,2021-05-02 20:00:00,250,0
3,A002 - R051 - 02-00-00,2021-05-03,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,2021-05-03 20:00:00,472,1
4,A002 - R051 - 02-00-00,2021-05-04,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,2021-05-04 20:00:00,501,1
5,A002 - R051 - 02-00-00,2021-05-05,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,2021-05-05 20:00:00,522,1


,ID,DATE,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DESC,DATETIME,NET_DAILY_ENTRIES,WEEKDAY
308641,TRAM2 - R469 - 00-05-01,2021-06-26,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,REGULAR,2021-06-26 21:00:00,0,0
308642,TRAM2 - R469 - 00-05-01,2021-06-27,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,REGULAR,2021-06-27 21:00:00,0,0
308643,TRAM2 - R469 - 00-05-01,2021-06-28,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,REGULAR,2021-06-28 21:00:00,0,1
308644,TRAM2 - R469 - 00-05-01,2021-06-29,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,REGULAR,2021-06-29 21:00:00,0,1
308645,TRAM2 - R469 - 00-05-01,2021-06-30,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,REGULAR,2021-06-30 21:00:00,0,1


### 2e) Figure out which turnstiles all belong to the same station and sum up station-level NET_DAILY_ENTRIES

#### While UNIT ("Remote Unit") generally denotes a station, certain stations actually have several UNITs associated with them; the term "Complex" describes the physical location where a Remote Unit resides.

Import MTA Complex List Dataset (based on MTA-provided data & augmented slightly by Chris Whong); found at the following link:

https://qri.cloud/nyc-transit-data/remote_complex_lookup

In [33]:
# NOTE: The 'remote' column in this dataset refers to the same entity as 'UNIT' in the turnstile data
#         (the full name is technically 'Remote Unit'); Also, 'booth' below refers to the same entity
#         as 'C/A' in the turnstile data

complex_list = pd.read_csv('nyc_transit_data_remote_complex_lookup.csv')

# Drop the division='SRT' rows (these are Staten Island Railway stations; not under consideration here);
#   Also drop 'LIB' and 'BEE' divisions (they are not included in the MTA Turnstile data)
complex_list = complex_list[~complex_list['division'].isin(['SRT', 'LIB', 'BEE'])].reset_index(drop=True)

print(complex_list.shape)
complex_list

(770, 6)


,remote,booth,complex_id,station,line_name,division
0,R001,A060,635.0,WHITEHALL ST,R1,BMT
1,R001,A058,635.0,WHITEHALL ST,R1,BMT
2,R001,R101S,635.0,SOUTH FERRY,R1,IRT
3,R002,A077,628.0,FULTON ST,ACJZ2345,BMT
4,R002,A081,628.0,FULTON ST,ACJZ2345,BMT
5,R002,A082,628.0,FULTON ST,ACJZ2345,BMT
6,R003,J025,86.0,CYPRESS HILLS,J,BMT
7,R004,J028,85.0,ELDERTS LANE,JZ,BMT
8,R005,J030,84.0,FOREST PARKWAY,J,BMT
9,R006,J031,83.0,WOODHAVEN BLVD,JZ,BMT


In [34]:
complex_list['division'].unique()

array(['BMT', 'IRT', 'IND', 'RIT', 'PTH'], dtype=object)

While we're at it, let's drop 'SRT' division from the daily_per_turnstile data (not interested in Staten Island)

In [35]:
print(daily_per_turnstile_2019.shape)
daily_per_turnstile_2019 = daily_per_turnstile_2019[daily_per_turnstile_2019['DIVISION'] != 'SRT']
daily_per_turnstile_2019.reset_index(drop=True, inplace=True)
print(daily_per_turnstile_2019.shape)

(295338, 12)
(293325, 12)


In [36]:
daily_per_turnstile_2019['DIVISION'].unique()

array(['BMT', 'IND', 'PTH', 'IRT', 'RIT'], dtype=object)

In [37]:
print(daily_per_turnstile_2021.shape)
daily_per_turnstile_2021 = daily_per_turnstile_2021[daily_per_turnstile_2021['DIVISION'] != 'SRT']
daily_per_turnstile_2021.reset_index(drop=True, inplace=True)
print(daily_per_turnstile_2021.shape)

(303670, 12)
(301718, 12)


In [38]:
daily_per_turnstile_2021['DIVISION'].unique()

array(['BMT', 'IND', 'PTH', 'IRT', 'RIT'], dtype=object)

Now import the dataset that maps complex_id to physical location of each "Complex" (MTA-provided data but again slightly modified by Chris Whong). Can be found at the following link: 

https://qri.cloud/nyc-transit-data/turnstiles_station_list

In [39]:
station_locations = pd.read_csv('nyc_transit_turnstiles_station_list.csv')

# Drop the division='SIR' rows (these are Staten Island Railway stations; not under consideration here)
station_locations = station_locations[station_locations['division'] != 'SIR'].reset_index(drop=True)

print(station_locations.shape)
station_locations

(490, 14)


,ogc_fid,station_id,complex_id,gtfs_stop_id,division,line,stop_name,borough,daytime_routes,structure,gtfs_latitude,gtfs_longitude,north_direction_label,south_direction_label
0,1,1.0,1,R01,BMT,Astoria,Astoria - Ditmars Blvd,Q,N W,Elevated,40.775036,-73.912034,NaN,Manhattan
1,2,2.0,2,R03,BMT,Astoria,Astoria Blvd,Q,N W,Elevated,40.770258,-73.917843,Ditmars Blvd,Manhattan
2,3,3.0,3,R04,BMT,Astoria,30 Av,Q,N W,Elevated,40.766779,-73.921479,Astoria - Ditmars Blvd,Manhattan
3,4,4.0,4,R05,BMT,Astoria,Broadway,Q,N W,Elevated,40.761820,-73.925508,Astoria - Ditmars Blvd,Manhattan
4,5,5.0,5,R06,BMT,Astoria,36 Av,Q,N W,Elevated,40.756804,-73.929575,Astoria - Ditmars Blvd,Manhattan
5,6,6.0,6,R08,BMT,Astoria,39 Av,Q,N W,Elevated,40.752882,-73.932755,Astoria - Ditmars Blvd,Manhattan
6,7,7.0,613,R11,BMT,Astoria,Lexington Av/59 St,M,N W R,Subway,40.762660,-73.967258,Queens,Downtown & Brooklyn
7,8,8.0,8,R13,BMT,Astoria,5 Av/59 St,M,N W R,Subway,40.764811,-73.973347,Queens,Downtown & Brooklyn
8,9,9.0,9,R14,BMT,Broadway - Brighton,57 St - 7 Av,M,N Q R W,Subway,40.764664,-73.980658,Uptown & Queens,Downtown & Brooklyn
9,10,10.0,10,R15,BMT,Broadway - Brighton,49 St,M,N R W,Subway,40.759901,-73.984139,Uptown & Queens,Downtown & Brooklyn


In [40]:
# Let's drop some unnecessary columns from station_locations

station_locations = station_locations[['complex_id', 'division', 'line', 'stop_name', 
                                       'borough', 'gtfs_latitude', 'gtfs_longitude']]
station_locations.head()

,complex_id,division,line,stop_name,borough,gtfs_latitude,gtfs_longitude
0,1,BMT,Astoria,Astoria - Ditmars Blvd,Q,40.775036,-73.912034
1,2,BMT,Astoria,Astoria Blvd,Q,40.770258,-73.917843
2,3,BMT,Astoria,30 Av,Q,40.766779,-73.921479
3,4,BMT,Astoria,Broadway,Q,40.761820,-73.925508
4,5,BMT,Astoria,36 Av,Q,40.756804,-73.929575


#### Now merge complex_list with station_locations

In [41]:
station_locations['complex_id'].dtype

dtype('O')

In [42]:
# Need to convert 'complex_id' in complex_list to same format

print(complex_list['complex_id'].dtype)

complex_list['complex_id'] = complex_list['complex_id'].apply(lambda x: str(x).replace('.0','')).astype('str')
print(complex_list['complex_id'].dtype)

float64
object


In [43]:
print(complex_list.shape)
complex_locations = complex_list.merge(right=station_locations, 
                                       how='inner', 
                                       on=['complex_id', 'division'])

(770, 6)


In [44]:
print(complex_locations.shape)

# Check for NaNs
print(complex_locations[complex_locations.isna().any(axis=1)].shape)

complex_locations.sample(10)

(809, 11)
(0, 11)


,remote,booth,complex_id,station,line_name,division,line,stop_name,borough,gtfs_latitude,gtfs_longitude
454,R205,R261,603,149 ST-GR CONC,245,IRT,Jerome Av,149 St - Grand Concourse,Bx,40.818375,-73.927351
108,R035,N012,605,168 ST-BROADWAY,1AC,IND,8th Av - Fulton St,168 St,M,40.840719,-73.939561
558,R276,R504,464,VERNON/JACKSON,7,IRT,Flushing,Vernon Blvd - Jackson Av,Q,40.742626,-73.953581
97,R033,R150,611,42 ST-TIMES SQ,1237ACENQRS,IRT,Flushing,Times Sq - 42 St,M,40.755477,-73.987691
516,R249,H014,122,GRAHAM AVE,L,BMT,Canarsie,Graham Av,Bk,40.714565,-73.944053
512,R454,C015,30,PROSPECT AVE,R,BMT,4th Av,Prospect Av,Bk,40.665414,-73.992872
344,R154,N213,215,TREMONT AVE,BD,IND,Concourse,Tremont Av,Bx,40.850410,-73.905227
273,R113,N300,277,7 AVE-53 ST,BDE,IND,Queens Blvd,7 Av,M,40.762862,-73.981637
438,R198,N116,179,NOSTRAND AVE,AC,IND,8th Av - Fulton St,Nostrand Av,Bk,40.680438,-73.950426
136,R046,R237,610,42 ST-GRD CNTRL,4567S,IRT,Flushing,Grand Central - 42 St,M,40.751431,-73.976041


#### Now we can merge complex_locations with the daily_per_turnstile datasets for 2019 & 2021

In [45]:
complex_locations.columns = [col.upper() for col in complex_locations.columns]
complex_locations.head()

,REMOTE,BOOTH,COMPLEX_ID,STATION,LINE_NAME,DIVISION,LINE,STOP_NAME,BOROUGH,GTFS_LATITUDE,GTFS_LONGITUDE
0,R001,A060,635,WHITEHALL ST,R1,BMT,Broadway,Whitehall St,M,40.703087,-74.012994
1,R001,A058,635,WHITEHALL ST,R1,BMT,Broadway,Whitehall St,M,40.703087,-74.012994
2,R001,R101S,635,SOUTH FERRY,R1,IRT,Broadway - 7Av,South Ferry,M,40.702068,-74.013664
3,R002,A077,628,FULTON ST,ACJZ2345,BMT,Jamaica,Fulton St,M,40.710374,-74.007582
4,R002,A081,628,FULTON ST,ACJZ2345,BMT,Jamaica,Fulton St,M,40.710374,-74.007582


In [48]:
# Drop unnecessary column
daily_per_turnstile_2019.drop(columns=['DATETIME'], inplace=True)

# Get consistent column names between the two dataframes to be merged

daily_per_turnstile_2019.columns = ['ID', 'DATE', 'BOOTH', 'REMOTE', 'SCP', 'STATION', 
                                    'LINE_NAME', 'DIVISION', 'DESC', 'NET_DAILY_ENTRIES', 'WEEKDAY']
daily_per_turnstile_2019.head()

,ID,DATE,BOOTH,REMOTE,SCP,STATION,LINE_NAME,DIVISION,DESC,NET_DAILY_ENTRIES,WEEKDAY
0,A002 - R051 - 02-00-00,2019-05-01,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,1346,1
1,A002 - R051 - 02-00-00,2019-05-02,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,1327,1
2,A002 - R051 - 02-00-00,2019-05-03,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,1326,1
3,A002 - R051 - 02-00-00,2019-05-04,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,560,0
4,A002 - R051 - 02-00-00,2019-05-05,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,449,0


In [49]:
daily_per_turnstile_loc_2019 = complex_locations.merge(right=daily_per_turnstile_2019, 
                                                       how='inner',
                                                       on=['BOOTH', 'REMOTE', 'DIVISION'])

In [50]:
# Didn't even lose 3000 rows here - pretty good!

print(daily_per_turnstile_2019.shape)
daily_per_turnstile_loc_2019.shape

(293325, 11)


(290643, 19)

In [51]:
# Check for NaNs

daily_per_turnstile_loc_2019[daily_per_turnstile_loc_2019.isna().any(axis=1)].shape[0]

0

In [52]:
# STATION_x and STATION_y seem to match up well! (LINE_NAME_x and LINE_NAME_y, too!)

daily_per_turnstile_loc_2019.sample(n=20)

,REMOTE,BOOTH,COMPLEX_ID,STATION_x,LINE_NAME_x,DIVISION,LINE,STOP_NAME,BOROUGH,GTFS_LATITUDE,GTFS_LONGITUDE,ID,DATE,SCP,STATION_y,LINE_NAME_y,DESC,NET_DAILY_ENTRIES,WEEKDAY
11440,R014,R208,628,FULTON ST,2345ACJZ,IRT,Clark St,Fulton St,M,40.709416,-74.006571,R208 - R014 - 03-01-02,2019-05-08,03-01-02,FULTON ST,2345ACJZ,REGULAR,441,1
139829,R141,N333,261,FOREST HILLS-71,EFMR,IND,Queens Blvd,Forest Hills - 71 Av,Q,40.721691,-73.844521,N333 - R141 - 01-00-02,2019-05-18,01-00-02,FOREST HILLS 71,EFMR,REGULAR,235,0
76055,R057,C009,617,PACIFIC ST,BDNQR2345,BMT,4th Av,Atlantic Av - Barclays Ctr,Bk,40.683666,-73.978810,C009 - R057 - 03-03-02,2019-05-31,03-03-02,ATL AV-BARCLAY,BDNQR2345,REGULAR,1532,1
17455,R018,R525,616,74 ST-BROADWAY,EFMR7,IRT,Flushing,74 St - Broadway,Q,40.746848,-73.891394,R525 - R018 - 02-00-02,2019-06-14,02-00-02,74 ST-BROADWAY,7EFMR,REGULAR,1566,1
134024,R137,H027,630,MYRTLE AVE,LM,BMT,Canarsie,Myrtle - Wyckoff Avs,Bk,40.699814,-73.911586,H027 - R137 - 01-06-00,2019-05-08,01-06-00,MYRTLE-WYCKOFF,LM,REGULAR,2074,1
151237,R158,N335,259,UNION TPK-KEW G,EF,IND,Queens Blvd,Kew Gardens - Union Tpke,Q,40.714441,-73.831008,N335 - R158 - 01-00-01,2019-05-06,01-00-01,KEW GARDENS,EF,REGULAR,1682,1
231991,R305,R106,328,CORTLANDT ST,1,IRT,Broadway - 7Av,WTC Cortlandt,M,40.711835,-74.012188,R106 - R305 - 01-00-01,2019-06-07,01-00-01,WTC-CORTLANDT,1,REGULAR,0,1
159179,R170,A035,602,14 ST-UNION SQ,LNQR456,BMT,Broadway - Brighton,14 St - Union Sq,M,40.735736,-73.990568,A035 - R170 - 00-00-03,2019-05-18,00-00-03,14 ST-UNION SQ,LNQR456W,REGULAR,881,0
63340,R047,R240,610,42 ST-GRD CNTRL,4567S,IRT,Flushing,Grand Central - 42 St,M,40.751431,-73.976041,R240 - R047 - 00-03-00,2019-06-14,00-03-00,GRD CNTRL-42 ST,4567S,REGULAR,2211,1
17402,R018,R525,616,74 ST-BROADWAY,EFMR7,IRT,Flushing,74 St - Broadway,Q,40.746848,-73.891394,R525 - R018 - 02-00-01,2019-06-22,02-00-01,74 ST-BROADWAY,7EFMR,REGULAR,662,0


#### Reorder the columns for more readability (might be useful not to drop any for future troubleshooting)

In [53]:
daily_per_turnstile_loc_2019 = daily_per_turnstile_loc_2019[['STATION_x', 'STATION_y', 'STOP_NAME', 
                                                             'LINE_NAME_x', 'LINE_NAME_y', 'DIVISION', 
                                                             'BOROUGH', 'DATE', 'NET_DAILY_ENTRIES', 'WEEKDAY', 
                                                             'DESC', 'GTFS_LATITUDE', 'GTFS_LONGITUDE', 'LINE', 
                                                             'REMOTE', 'BOOTH', 'SCP', 'COMPLEX_ID', 'ID']]
print(daily_per_turnstile_loc_2019.shape)
daily_per_turnstile_loc_2019.head()

(290643, 19)


,STATION_x,STATION_y,STOP_NAME,LINE_NAME_x,LINE_NAME_y,DIVISION,BOROUGH,DATE,NET_DAILY_ENTRIES,WEEKDAY,DESC,GTFS_LATITUDE,GTFS_LONGITUDE,LINE,REMOTE,BOOTH,SCP,COMPLEX_ID,ID
0,WHITEHALL ST,WHITEHALL S-FRY,Whitehall St,R1,R1W,BMT,M,2019-05-01,2604,1,REGULAR,40.703087,-74.012994,Broadway,R001,A060,00-00-00,635,A060 - R001 - 00-00-00
1,WHITEHALL ST,WHITEHALL S-FRY,Whitehall St,R1,R1W,BMT,M,2019-05-02,2676,1,REGULAR,40.703087,-74.012994,Broadway,R001,A060,00-00-00,635,A060 - R001 - 00-00-00
2,WHITEHALL ST,WHITEHALL S-FRY,Whitehall St,R1,R1W,BMT,M,2019-05-03,2768,1,REGULAR,40.703087,-74.012994,Broadway,R001,A060,00-00-00,635,A060 - R001 - 00-00-00
3,WHITEHALL ST,WHITEHALL S-FRY,Whitehall St,R1,R1W,BMT,M,2019-05-04,1717,0,REGULAR,40.703087,-74.012994,Broadway,R001,A060,00-00-00,635,A060 - R001 - 00-00-00
4,WHITEHALL ST,WHITEHALL S-FRY,Whitehall St,R1,R1W,BMT,M,2019-05-05,1184,0,REGULAR,40.703087,-74.012994,Broadway,R001,A060,00-00-00,635,A060 - R001 - 00-00-00


##### Repeat all of the above steps for the 2021 data

In [54]:
# Drop unnecessary column
daily_per_turnstile_2021.drop(columns=['DATETIME'], inplace=True)

# Get consistent column names between the two dataframes to be merged

daily_per_turnstile_2021.columns = ['ID', 'DATE', 'BOOTH', 'REMOTE', 'SCP', 'STATION', 
                                    'LINE_NAME', 'DIVISION', 'DESC', 'NET_DAILY_ENTRIES', 'WEEKDAY']
daily_per_turnstile_2021.head()

,ID,DATE,BOOTH,REMOTE,SCP,STATION,LINE_NAME,DIVISION,DESC,NET_DAILY_ENTRIES,WEEKDAY
0,A002 - R051 - 02-00-00,2021-05-01,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,361,0
1,A002 - R051 - 02-00-00,2021-05-02,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,250,0
2,A002 - R051 - 02-00-00,2021-05-03,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,472,1
3,A002 - R051 - 02-00-00,2021-05-04,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,501,1
4,A002 - R051 - 02-00-00,2021-05-05,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,522,1


In [55]:
daily_per_turnstile_loc_2021 = complex_locations.merge(right=daily_per_turnstile_2021, 
                                                       how='inner',
                                                       on=['BOOTH', 'REMOTE', 'DIVISION'])

In [56]:
# Again, only lost ~3500 rows here - pretty good!

print(daily_per_turnstile_2021.shape)
daily_per_turnstile_loc_2021.shape

(301718, 11)


(298140, 19)

In [57]:
# Check for NaNs

daily_per_turnstile_loc_2021[daily_per_turnstile_loc_2021.isna().any(axis=1)].shape[0]

0

In [58]:
# Again, STATION_x and STATION_y seem to match up well! (LINE_NAME_x and LINE_NAME_y, too!)

daily_per_turnstile_loc_2021.sample(n=20)

,REMOTE,BOOTH,COMPLEX_ID,STATION_x,LINE_NAME_x,DIVISION,LINE,STOP_NAME,BOROUGH,GTFS_LATITUDE,GTFS_LONGITUDE,ID,DATE,SCP,STATION_y,LINE_NAME_y,DESC,NET_DAILY_ENTRIES,WEEKDAY
266621,R377,J007,98,FLUSHING AVE,JM,BMT,Jamaica,Flushing Av,Bk,40.700260,-73.941126,J007 - R377 - 00-05-00,2021-05-28,00-05-00,FLUSHING AV,JM,REGULAR,0,1
226567,R282,N087,168,SPRING ST,CE,IND,8th Av - Fulton St,Spring St,M,40.726227,-74.003739,N087 - R282 - 01-06-00,2021-05-14,01-06-00,SPRING ST,CE,REGULAR,509,1
23254,R022,N506,607,34 ST-HERALD SQ,BDFMNQR,IND,6th Av - Culver,34 St - Herald Sq,M,40.749719,-73.987823,N506 - R022 - 00-05-00,2021-06-02,00-05-00,34 ST-HERALD SQ,BDFMNQRW,REGULAR,340,1
156375,R162,R414,369,ELDER AVE,6,IRT,Pelham,Elder Av,Bx,40.828584,-73.879159,R414 - R162 - 00-03-01,2021-06-25,00-03-01,ELDER AV,6,REGULAR,1003,1
51153,R043,R204A,413,WALL ST,45,IRT,Lexington Av,Wall St,M,40.707557,-74.011862,R204A - R043 - 03-05-01,2021-05-09,03-05-01,WALL ST,45,REGULAR,0,0
180429,R193,R180,303,157 ST,1,IRT,Broadway - 7Av,157 St,M,40.834041,-73.944890,R180 - R193 - 00-00-03,2021-05-17,00-00-03,157 ST,1,REGULAR,237,1
81195,R064,R628,347,SARATOGA AVE,3,IRT,Eastern Pky,Saratoga Av,Bk,40.661453,-73.916327,R628 - R064 - 00-00-00,2021-05-26,00-00-00,SARATOGA AV,3,REGULAR,526,1
41883,R033,R151,611,42 ST-TIMES SQ,1237ACENQRS,IRT,Lexington - Shuttle,Times Sq - 42 St,M,40.755983,-73.986229,R151 - R033 - 00-00-01,2021-06-16,00-00-01,TIMES SQ-42 ST,1237ACENQRSW,REGULAR,1086,1
37670,R031,R139,318,34 ST-PENN STA,123,IRT,Broadway - 7Av,34 St - Penn Station,M,40.750373,-73.991057,R139 - R031 - 04-00-03,2021-06-12,04-00-03,34 ST-PENN STA,123,REGULAR,299,0
54835,R045,R236,610,42 ST-GRD CNTRL,4567S,IRT,Lexington - Shuttle,Grand Central - 42 St,M,40.752769,-73.979189,R236 - R045 - 00-00-02,2021-06-10,00-00-02,GRD CNTRL-42 ST,4567S,REGULAR,190,1


#### Reorder the columns for more readability (might be useful not to drop any for future troubleshooting)

In [59]:
daily_per_turnstile_loc_2021 = daily_per_turnstile_loc_2021[['STATION_x', 'STATION_y', 'STOP_NAME', 
                                                             'LINE_NAME_x', 'LINE_NAME_y', 'DIVISION', 
                                                             'BOROUGH', 'DATE', 'NET_DAILY_ENTRIES', 'WEEKDAY', 
                                                             'DESC', 'GTFS_LATITUDE', 'GTFS_LONGITUDE', 'LINE', 
                                                             'REMOTE', 'BOOTH', 'SCP', 'COMPLEX_ID', 'ID']]
print(daily_per_turnstile_loc_2021.shape)
daily_per_turnstile_loc_2021.head()

(298140, 19)


,STATION_x,STATION_y,STOP_NAME,LINE_NAME_x,LINE_NAME_y,DIVISION,BOROUGH,DATE,NET_DAILY_ENTRIES,WEEKDAY,DESC,GTFS_LATITUDE,GTFS_LONGITUDE,LINE,REMOTE,BOOTH,SCP,COMPLEX_ID,ID
0,WHITEHALL ST,WHITEHALL S-FRY,Whitehall St,R1,R1W,BMT,M,2021-05-01,692,0,REGULAR,40.703087,-74.012994,Broadway,R001,A060,00-00-00,635,A060 - R001 - 00-00-00
1,WHITEHALL ST,WHITEHALL S-FRY,Whitehall St,R1,R1W,BMT,M,2021-05-02,578,0,REGULAR,40.703087,-74.012994,Broadway,R001,A060,00-00-00,635,A060 - R001 - 00-00-00
2,WHITEHALL ST,WHITEHALL S-FRY,Whitehall St,R1,R1W,BMT,M,2021-05-03,895,1,REGULAR,40.703087,-74.012994,Broadway,R001,A060,00-00-00,635,A060 - R001 - 00-00-00
3,WHITEHALL ST,WHITEHALL S-FRY,Whitehall St,R1,R1W,BMT,M,2021-05-04,947,1,REGULAR,40.703087,-74.012994,Broadway,R001,A060,00-00-00,635,A060 - R001 - 00-00-00
4,WHITEHALL ST,WHITEHALL S-FRY,Whitehall St,R1,R1W,BMT,M,2021-05-05,905,1,REGULAR,40.703087,-74.012994,Broadway,R001,A060,00-00-00,635,A060 - R001 - 00-00-00


### 2f) Now generate NET_DAILY_STATION_ENTRIES (i.e. aggregate from per-turnstile to per-station basis)

#### (Recall from previous section that 'COMPLEX_ID' is the unifying element that identifies all turnstiles residing "under one roof")

In [60]:
daily_per_station_2019 = daily_per_turnstile_loc_2019.groupby(['COMPLEX_ID', 'STATION_x', 'DIVISION',
                                                               'DATE'])['NET_DAILY_ENTRIES'] \
                                                     .sum().reset_index()

daily_per_station_2019.columns = ['COMPLEX_ID', 'STATION_x', 'DIVISION', 'DATE', 'NET_DAILY_STATION_ENTRIES']
print(daily_per_station_2019.shape)
display(daily_per_station_2019.head())
display(daily_per_station_2019.tail())

(27512, 5)


,COMPLEX_ID,STATION_x,DIVISION,DATE,NET_DAILY_STATION_ENTRIES
0,1,DITMARS BL-31 S,BMT,2019-05-01,18999
1,1,DITMARS BL-31 S,BMT,2019-05-02,19257
2,1,DITMARS BL-31 S,BMT,2019-05-03,18661
3,1,DITMARS BL-31 S,BMT,2019-05-04,9577
4,1,DITMARS BL-31 S,BMT,2019-05-05,6630


,COMPLEX_ID,STATION_x,DIVISION,DATE,NET_DAILY_STATION_ENTRIES
27507,99,LORIMER ST,BMT,2019-06-26,5117
27508,99,LORIMER ST,BMT,2019-06-27,5155
27509,99,LORIMER ST,BMT,2019-06-28,4908
27510,99,LORIMER ST,BMT,2019-06-29,3587
27511,99,LORIMER ST,BMT,2019-06-30,3284


#### Now merge in the other desired columns

In [61]:
daily_per_station_2019 = daily_per_station_2019.merge(right=daily_per_turnstile_loc_2019
                                                      .groupby(['COMPLEX_ID', 'STATION_x', 'DIVISION',
                                                                'DATE'])
                                                      .first().reset_index(),
                                                      how='inner',
                                                      on=['COMPLEX_ID', 'STATION_x', 'DIVISION', 'DATE']) \
                                                      .drop(columns='NET_DAILY_ENTRIES')

In [62]:
print(daily_per_station_2019.shape)
display(daily_per_station_2019.head(10))
display(daily_per_station_2019.tail(10))

(27512, 19)


,COMPLEX_ID,STATION_x,DIVISION,DATE,NET_DAILY_STATION_ENTRIES,STATION_y,STOP_NAME,LINE_NAME_x,LINE_NAME_y,BOROUGH,WEEKDAY,DESC,GTFS_LATITUDE,GTFS_LONGITUDE,LINE,REMOTE,BOOTH,SCP,ID
0,1,DITMARS BL-31 S,BMT,2019-05-01,18999,ASTORIA DITMARS,Astoria - Ditmars Blvd,NQ,NQW,Q,1,REGULAR,40.775036,-73.912034,Astoria,R095,R515,00-00-00,R515 - R095 - 00-00-00
1,1,DITMARS BL-31 S,BMT,2019-05-02,19257,ASTORIA DITMARS,Astoria - Ditmars Blvd,NQ,NQW,Q,1,REGULAR,40.775036,-73.912034,Astoria,R095,R515,00-00-00,R515 - R095 - 00-00-00
2,1,DITMARS BL-31 S,BMT,2019-05-03,18661,ASTORIA DITMARS,Astoria - Ditmars Blvd,NQ,NQW,Q,1,REGULAR,40.775036,-73.912034,Astoria,R095,R515,00-00-00,R515 - R095 - 00-00-00
3,1,DITMARS BL-31 S,BMT,2019-05-04,9577,ASTORIA DITMARS,Astoria - Ditmars Blvd,NQ,NQW,Q,0,REGULAR,40.775036,-73.912034,Astoria,R095,R515,00-00-00,R515 - R095 - 00-00-00
4,1,DITMARS BL-31 S,BMT,2019-05-05,6630,ASTORIA DITMARS,Astoria - Ditmars Blvd,NQ,NQW,Q,0,REGULAR,40.775036,-73.912034,Astoria,R095,R515,00-00-00,R515 - R095 - 00-00-00
5,1,DITMARS BL-31 S,BMT,2019-05-06,18272,ASTORIA DITMARS,Astoria - Ditmars Blvd,NQ,NQW,Q,1,REGULAR,40.775036,-73.912034,Astoria,R095,R515,00-00-00,R515 - R095 - 00-00-00
6,1,DITMARS BL-31 S,BMT,2019-05-07,19146,ASTORIA DITMARS,Astoria - Ditmars Blvd,NQ,NQW,Q,1,REGULAR,40.775036,-73.912034,Astoria,R095,R515,00-00-00,R515 - R095 - 00-00-00
7,1,DITMARS BL-31 S,BMT,2019-05-08,19359,ASTORIA DITMARS,Astoria - Ditmars Blvd,NQ,NQW,Q,1,REGULAR,40.775036,-73.912034,Astoria,R095,R515,00-00-00,R515 - R095 - 00-00-00
8,1,DITMARS BL-31 S,BMT,2019-05-09,19073,ASTORIA DITMARS,Astoria - Ditmars Blvd,NQ,NQW,Q,1,REGULAR,40.775036,-73.912034,Astoria,R095,R515,00-00-00,R515 - R095 - 00-00-00
9,1,DITMARS BL-31 S,BMT,2019-05-10,18527,ASTORIA DITMARS,Astoria - Ditmars Blvd,NQ,NQW,Q,1,REGULAR,40.775036,-73.912034,Astoria,R095,R515,00-00-00,R515 - R095 - 00-00-00


,COMPLEX_ID,STATION_x,DIVISION,DATE,NET_DAILY_STATION_ENTRIES,STATION_y,STOP_NAME,LINE_NAME_x,LINE_NAME_y,BOROUGH,WEEKDAY,DESC,GTFS_LATITUDE,GTFS_LONGITUDE,LINE,REMOTE,BOOTH,SCP,ID
27502,99,LORIMER ST,BMT,2019-06-21,5253,LORIMER ST,Lorimer St,JM,JM,Bk,1,REGULAR,40.703869,-73.947408,Jamaica,R353,J005,00-00-00,J005 - R353 - 00-00-00
27503,99,LORIMER ST,BMT,2019-06-22,4194,LORIMER ST,Lorimer St,JM,JM,Bk,0,RECOVR AUD,40.703869,-73.947408,Jamaica,R353,J005,00-00-00,J005 - R353 - 00-00-00
27504,99,LORIMER ST,BMT,2019-06-23,3647,LORIMER ST,Lorimer St,JM,JM,Bk,0,REGULAR,40.703869,-73.947408,Jamaica,R353,J005,00-00-00,J005 - R353 - 00-00-00
27505,99,LORIMER ST,BMT,2019-06-24,5175,LORIMER ST,Lorimer St,JM,JM,Bk,1,REGULAR,40.703869,-73.947408,Jamaica,R353,J005,00-00-00,J005 - R353 - 00-00-00
27506,99,LORIMER ST,BMT,2019-06-25,5099,LORIMER ST,Lorimer St,JM,JM,Bk,1,REGULAR,40.703869,-73.947408,Jamaica,R353,J005,00-00-00,J005 - R353 - 00-00-00
27507,99,LORIMER ST,BMT,2019-06-26,5117,LORIMER ST,Lorimer St,JM,JM,Bk,1,REGULAR,40.703869,-73.947408,Jamaica,R353,J005,00-00-00,J005 - R353 - 00-00-00
27508,99,LORIMER ST,BMT,2019-06-27,5155,LORIMER ST,Lorimer St,JM,JM,Bk,1,REGULAR,40.703869,-73.947408,Jamaica,R353,J005,00-00-00,J005 - R353 - 00-00-00
27509,99,LORIMER ST,BMT,2019-06-28,4908,LORIMER ST,Lorimer St,JM,JM,Bk,1,REGULAR,40.703869,-73.947408,Jamaica,R353,J005,00-00-00,J005 - R353 - 00-00-00
27510,99,LORIMER ST,BMT,2019-06-29,3587,LORIMER ST,Lorimer St,JM,JM,Bk,0,REGULAR,40.703869,-73.947408,Jamaica,R353,J005,00-00-00,J005 - R353 - 00-00-00
27511,99,LORIMER ST,BMT,2019-06-30,3284,LORIMER ST,Lorimer St,JM,JM,Bk,0,REGULAR,40.703869,-73.947408,Jamaica,R353,J005,00-00-00,J005 - R353 - 00-00-00


##### Do the same for 2021 data

In [63]:
daily_per_station_2021 = daily_per_turnstile_loc_2021.groupby(['COMPLEX_ID', 'STATION_x', 'DIVISION',
                                                               'DATE'])['NET_DAILY_ENTRIES'] \
                                                     .sum().reset_index()

daily_per_station_2021.columns = ['COMPLEX_ID', 'STATION_x', 'DIVISION', 'DATE', 'NET_DAILY_STATION_ENTRIES']
print(daily_per_station_2021.shape)
display(daily_per_station_2021.head())
display(daily_per_station_2021.tail())

(27567, 5)


,COMPLEX_ID,STATION_x,DIVISION,DATE,NET_DAILY_STATION_ENTRIES
0,1,DITMARS BL-31 S,BMT,2021-05-01,3485
1,1,DITMARS BL-31 S,BMT,2021-05-02,2378
2,1,DITMARS BL-31 S,BMT,2021-05-03,4990
3,1,DITMARS BL-31 S,BMT,2021-05-04,5290
4,1,DITMARS BL-31 S,BMT,2021-05-05,5196


,COMPLEX_ID,STATION_x,DIVISION,DATE,NET_DAILY_STATION_ENTRIES
27562,99,LORIMER ST,BMT,2021-06-26,1132
27563,99,LORIMER ST,BMT,2021-06-27,981
27564,99,LORIMER ST,BMT,2021-06-28,1663
27565,99,LORIMER ST,BMT,2021-06-29,1542
27566,99,LORIMER ST,BMT,2021-06-30,1669


#### Now merge in the other desired columns

In [64]:
daily_per_station_2021 = daily_per_station_2021.merge(right=daily_per_turnstile_loc_2021
                                                      .groupby(['COMPLEX_ID', 'STATION_x', 'DIVISION',
                                                                'DATE'])
                                                      .first().reset_index(),
                                                      how='inner',
                                                      on=['COMPLEX_ID', 'STATION_x', 'DIVISION', 'DATE']) \
                                                      .drop(columns='NET_DAILY_ENTRIES')

In [65]:
print(daily_per_station_2021.shape)
display(daily_per_station_2021.head(10))
display(daily_per_station_2021.tail(10))

(27567, 19)


,COMPLEX_ID,STATION_x,DIVISION,DATE,NET_DAILY_STATION_ENTRIES,STATION_y,STOP_NAME,LINE_NAME_x,LINE_NAME_y,BOROUGH,WEEKDAY,DESC,GTFS_LATITUDE,GTFS_LONGITUDE,LINE,REMOTE,BOOTH,SCP,ID
0,1,DITMARS BL-31 S,BMT,2021-05-01,3485,ASTORIA DITMARS,Astoria - Ditmars Blvd,NQ,NQW,Q,0,REGULAR,40.775036,-73.912034,Astoria,R095,R515,00-00-00,R515 - R095 - 00-00-00
1,1,DITMARS BL-31 S,BMT,2021-05-02,2378,ASTORIA DITMARS,Astoria - Ditmars Blvd,NQ,NQW,Q,0,REGULAR,40.775036,-73.912034,Astoria,R095,R515,00-00-00,R515 - R095 - 00-00-00
2,1,DITMARS BL-31 S,BMT,2021-05-03,4990,ASTORIA DITMARS,Astoria - Ditmars Blvd,NQ,NQW,Q,1,REGULAR,40.775036,-73.912034,Astoria,R095,R515,00-00-00,R515 - R095 - 00-00-00
3,1,DITMARS BL-31 S,BMT,2021-05-04,5290,ASTORIA DITMARS,Astoria - Ditmars Blvd,NQ,NQW,Q,1,REGULAR,40.775036,-73.912034,Astoria,R095,R515,00-00-00,R515 - R095 - 00-00-00
4,1,DITMARS BL-31 S,BMT,2021-05-05,5196,ASTORIA DITMARS,Astoria - Ditmars Blvd,NQ,NQW,Q,1,REGULAR,40.775036,-73.912034,Astoria,R095,R515,00-00-00,R515 - R095 - 00-00-00
5,1,DITMARS BL-31 S,BMT,2021-05-06,5470,ASTORIA DITMARS,Astoria - Ditmars Blvd,NQ,NQW,Q,1,REGULAR,40.775036,-73.912034,Astoria,R095,R515,00-00-00,R515 - R095 - 00-00-00
6,1,DITMARS BL-31 S,BMT,2021-05-07,5448,ASTORIA DITMARS,Astoria - Ditmars Blvd,NQ,NQW,Q,1,REGULAR,40.775036,-73.912034,Astoria,R095,R515,00-00-00,R515 - R095 - 00-00-00
7,1,DITMARS BL-31 S,BMT,2021-05-08,3192,ASTORIA DITMARS,Astoria - Ditmars Blvd,NQ,NQW,Q,0,REGULAR,40.775036,-73.912034,Astoria,R095,R515,00-00-00,R515 - R095 - 00-00-00
8,1,DITMARS BL-31 S,BMT,2021-05-09,2362,ASTORIA DITMARS,Astoria - Ditmars Blvd,NQ,NQW,Q,0,REGULAR,40.775036,-73.912034,Astoria,R095,R515,00-00-00,R515 - R095 - 00-00-00
9,1,DITMARS BL-31 S,BMT,2021-05-10,4986,ASTORIA DITMARS,Astoria - Ditmars Blvd,NQ,NQW,Q,1,REGULAR,40.775036,-73.912034,Astoria,R095,R515,00-00-00,R515 - R095 - 00-00-00


,COMPLEX_ID,STATION_x,DIVISION,DATE,NET_DAILY_STATION_ENTRIES,STATION_y,STOP_NAME,LINE_NAME_x,LINE_NAME_y,BOROUGH,WEEKDAY,DESC,GTFS_LATITUDE,GTFS_LONGITUDE,LINE,REMOTE,BOOTH,SCP,ID
27557,99,LORIMER ST,BMT,2021-06-21,1711,LORIMER ST,Lorimer St,JM,JM,Bk,1,REGULAR,40.703869,-73.947408,Jamaica,R353,J005,00-00-00,J005 - R353 - 00-00-00
27558,99,LORIMER ST,BMT,2021-06-22,1705,LORIMER ST,Lorimer St,JM,JM,Bk,1,REGULAR,40.703869,-73.947408,Jamaica,R353,J005,00-00-00,J005 - R353 - 00-00-00
27559,99,LORIMER ST,BMT,2021-06-23,1909,LORIMER ST,Lorimer St,JM,JM,Bk,1,REGULAR,40.703869,-73.947408,Jamaica,R353,J005,00-00-00,J005 - R353 - 00-00-00
27560,99,LORIMER ST,BMT,2021-06-24,1836,LORIMER ST,Lorimer St,JM,JM,Bk,1,REGULAR,40.703869,-73.947408,Jamaica,R353,J005,00-00-00,J005 - R353 - 00-00-00
27561,99,LORIMER ST,BMT,2021-06-25,1868,LORIMER ST,Lorimer St,JM,JM,Bk,1,REGULAR,40.703869,-73.947408,Jamaica,R353,J005,00-00-00,J005 - R353 - 00-00-00
27562,99,LORIMER ST,BMT,2021-06-26,1132,LORIMER ST,Lorimer St,JM,JM,Bk,0,REGULAR,40.703869,-73.947408,Jamaica,R353,J005,00-00-00,J005 - R353 - 00-00-00
27563,99,LORIMER ST,BMT,2021-06-27,981,LORIMER ST,Lorimer St,JM,JM,Bk,0,REGULAR,40.703869,-73.947408,Jamaica,R353,J005,00-00-00,J005 - R353 - 00-00-00
27564,99,LORIMER ST,BMT,2021-06-28,1663,LORIMER ST,Lorimer St,JM,JM,Bk,1,REGULAR,40.703869,-73.947408,Jamaica,R353,J005,00-00-00,J005 - R353 - 00-00-00
27565,99,LORIMER ST,BMT,2021-06-29,1542,LORIMER ST,Lorimer St,JM,JM,Bk,1,REGULAR,40.703869,-73.947408,Jamaica,R353,J005,00-00-00,J005 - R353 - 00-00-00
27566,99,LORIMER ST,BMT,2021-06-30,1669,LORIMER ST,Lorimer St,JM,JM,Bk,1,REGULAR,40.703869,-73.947408,Jamaica,R353,J005,00-00-00,J005 - R353 - 00-00-00


#### Verify that we've still got a common set of stations between the two years

In [66]:
print(daily_per_station_2019['STATION_x'].unique().shape)
print(daily_per_station_2021['STATION_x'].unique().shape)
print(np.setdiff1d(daily_per_station_2019['STATION_x'].unique(), daily_per_station_2021['STATION_x'].unique()))
print(np.setdiff1d(daily_per_station_2021['STATION_x'].unique(), daily_per_station_2019['STATION_x'].unique()))

(360,)
(360,)
[]
[]


### 2g) Now add ZIPCODEs corresponding to the latitude/longitude information

In [67]:
# Just using WEEKDAY column as a placeholder during grouping; will drop immediately

locations = daily_per_station_2019.groupby(['COMPLEX_ID', 'STATION_x', 'DIVISION', 
                                            'GTFS_LATITUDE', 'GTFS_LONGITUDE']) \
                                  ['WEEKDAY'].first().reset_index()

locations.drop(columns='WEEKDAY', inplace=True)
print(locations.shape)
locations

(452, 5)


,COMPLEX_ID,STATION_x,DIVISION,GTFS_LATITUDE,GTFS_LONGITUDE
0,1,DITMARS BL-31 S,BMT,40.775036,-73.912034
1,10,49 ST-7 AVE,BMT,40.759901,-73.984139
2,100,HEWES ST,BMT,40.706870,-73.953431
3,101,MARCY AVE,BMT,40.708359,-73.957757
4,103,BOWERY,BMT,40.720280,-73.993915
5,107,BROAD ST,BMT,40.706476,-74.011056
6,108,METROPOLITAN AV,BMT,40.711396,-73.889601
7,109,FRESH POND ROAD,BMT,40.706186,-73.895877
8,110,FOREST AVE,BMT,40.704423,-73.903077
9,111,SENECA AVE,BMT,40.702762,-73.907740


In [68]:
# This implementation is based on the info posted here:
#  https://stackoverflow.com/questions/66144427/how-do-i-get-zipcodes-from-longitude-and-latitude-on-python

# geo = geopy.Nominatim(user_agent="check_1")

def get_zipcode(df):
    geo = geopy.Nominatim(user_agent="check_1")
    try:
        return geo.reverse('{}, {}'.format(df['GTFS_LATITUDE'], df['GTFS_LONGITUDE'])) \
                                   .raw['address']['postcode']
    except:
        return np.nan

locations['ZIPCODE'] = locations.apply(get_zipcode, axis=1)
    
# locations['ZIPCODE'] = locations.apply(lambda x: geo.reverse('{}, {}'
#                                 .format(x['GTFS_LATITUDE'], x['GTFS_LONGITUDE'])) \
#                                 .raw['address']['postcode'], axis=1)

# geo.reverse("{}, {}".format(40.641362, -74.017881)).raw['address']['neighbourhood']

In [69]:
print(locations.shape)
display(locations[locations['ZIPCODE'].isna()])

(452, 6)


,COMPLEX_ID,STATION_x,DIVISION,GTFS_LATITUDE,GTFS_LONGITUDE,ZIPCODE
188,313,72 ST,IRT,40.778453,-73.98197,NaN


In [70]:
# Not sure why the coordinates above failed in Geopy; manual lookup reveals the ZIPCODE to be 10023

locations.loc[188, 'ZIPCODE'] = 10023
display(locations[locations['ZIPCODE'].isna()])

,COMPLEX_ID,STATION_x,DIVISION,GTFS_LATITUDE,GTFS_LONGITUDE,ZIPCODE


#### Strip off the -xxxx extension present in some of the ZIPCODEs

In [71]:
locations['ZIPCODE'] = locations['ZIPCODE'].apply(lambda x: str(x).split(sep = '-')[0])

locations[locations['ZIPCODE'].map(len) > 5]

,COMPLEX_ID,STATION_x,DIVISION,GTFS_LATITUDE,GTFS_LONGITUDE,ZIPCODE
316,448,METS-WILLETS PT,IRT,40.754622,-73.845625,111354


In [72]:
# Geopy fails again: Manual lookup of the coordinates above reveals ZIPCODE to be 11368

locations.loc[316, 'ZIPCODE'] = 11368
locations.loc[316, 'ZIPCODE']

11368

In [73]:
# Another known error (don't worry, Geopy - I still like you better than the GoogleMaps API)

locations[locations['ZIPCODE'] == '11227']

,COMPLEX_ID,STATION_x,DIVISION,GTFS_LATITUDE,GTFS_LONGITUDE,ZIPCODE
207,337,NEVINS ST,IRT,40.688246,-73.980492,11227


In [74]:
# 11227 is not a valid ZIPCODE; manual lookup of the coordinates above reveals ZIPCODE to be 11217

locations.loc[207, 'ZIPCODE'] = 11217
locations.loc[207, 'ZIPCODE']

11217

In [75]:
locations.head()

,COMPLEX_ID,STATION_x,DIVISION,GTFS_LATITUDE,GTFS_LONGITUDE,ZIPCODE
0,1,DITMARS BL-31 S,BMT,40.775036,-73.912034,11101
1,10,49 ST-7 AVE,BMT,40.759901,-73.984139,10019
2,100,HEWES ST,BMT,40.706870,-73.953431,11211
3,101,MARCY AVE,BMT,40.708359,-73.957757,11211
4,103,BOWERY,BMT,40.720280,-73.993915,10002


#### Add ZIPCODE column to daily_per_station datasets for both years

In [76]:
print(daily_per_station_2019.shape)
daily_per_station_2019 =daily_per_station_2019.merge(right=locations, 
                                                     how='left', 
                                                     on=['COMPLEX_ID', 'STATION_x', 'DIVISION',
                                                         'GTFS_LATITUDE', 'GTFS_LONGITUDE'])
print(daily_per_station_2019.shape)

(27512, 19)
(27512, 20)


In [77]:
daily_per_station_2019.sample(10)

,COMPLEX_ID,STATION_x,DIVISION,DATE,NET_DAILY_STATION_ENTRIES,STATION_y,STOP_NAME,LINE_NAME_x,LINE_NAME_y,BOROUGH,WEEKDAY,DESC,GTFS_LATITUDE,GTFS_LONGITUDE,LINE,REMOTE,BOOTH,SCP,ID,ZIPCODE
26227,8,49 ST-7 AVE,BMT,2019-05-20,11960,49 ST,5 Av/59 St,NQR,NQRW,M,1,REGULAR,40.764811,-73.973347,Astoria,R081,A015,00-00-00,A015 - R081 - 00-00-00,10019
2527,152,135 ST,IND,2019-06-05,5703,135 ST,135 St,BC,BC,M,1,REGULAR,40.817894,-73.947649,8th Av - Fulton St,R332,N024,00-00-00,N024 - R332 - 00-00-00,10040
9079,272,36 ST,IND,2019-06-30,349,36 ST,36 St,MR,MR,Q,0,REGULAR,40.752039,-73.928781,Queens Blvd,R339,N312,00-00-00,N312 - R339 - 00-00-00,11101
13418,350,PENNSYLVANIA AV,IRT,2019-05-10,4661,PENNSYLVANIA AV,Pennsylvania Av,3,3,Bk,1,REGULAR,40.664635,-73.894895,Eastern Pky,R067,R632,00-00-00,R632 - R067 - 00-00-00,11207
14412,365,CASTLE HILL AVE,IRT,2019-05-28,6104,CASTLE HILL AV,Castle Hill Av,6,6,Bx,1,REGULAR,40.834255,-73.851222,Pelham,R106,R418,00-00-00,R418 - R106 - 00-00-00,10461
3027,16,8 ST-B'WAY NYU,BMT,2019-06-17,12618,8 ST-NYU,8 St - NYU,NR,NRW,M,1,REGULAR,40.730328,-73.992629,Broadway - Brighton,R085,A038,00-00-00,A038 - R085 - 00-00-00,10003
12783,34,53 ST,BMT,2019-06-14,8979,53 ST,53 St,R,R,Bk,1,REGULAR,40.645069,-74.014034,4th Av,R233,C020,00-00-00,C020 - R233 - 00-00-00,11220
10151,292,FULTON ST,IND,2019-06-04,3790,FULTON ST,Fulton St,G,G,Bk,1,REGULAR,40.687119,-73.975375,Crosstown,R318,N422,00-00-00,N422 - R318 - 00-00-00,11221
7658,249,AVE P,IND,2019-06-12,2591,AVENUE P,Avenue P,F,F,Bk,1,REGULAR,40.608944,-73.973022,6th Av - Culver,R424,N556,00-00-00,N556 - R424 - 00-00-00,11223
1438,130,WILSON AVE,BMT,2019-06-11,4126,WILSON AV,Wilson Av,L,L,Bk,1,REGULAR,40.688764,-73.904046,Canarsie,R295,H032,00-00-00,H032 - R295 - 00-00-00,11207


#### Double check for NaNs, then save daily_per_station_2019 to file (can reload below and avoid running all of above in future)

In [78]:
daily_per_station_2019[daily_per_station_2019.isna().any(axis=1)].shape[0]

0

In [79]:
# Okay, write this dataframe to file and then we don't have to run any of the above again!

daily_per_station_2019.to_csv('daily_per_station_2019.csv', index=False)

##### Repeat the above steps for the 2021 data

In [80]:
# Add ZIPCODE column

print(daily_per_station_2021.shape)
daily_per_station_2021 =daily_per_station_2021.merge(right=locations, 
                                                     how='left', 
                                                     on=['COMPLEX_ID', 'STATION_x', 'DIVISION',
                                                         'GTFS_LATITUDE', 'GTFS_LONGITUDE'])
print(daily_per_station_2021.shape)

(27567, 19)
(27567, 20)


In [81]:
daily_per_station_2021.sample(10)

,COMPLEX_ID,STATION_x,DIVISION,DATE,NET_DAILY_STATION_ENTRIES,STATION_y,STOP_NAME,LINE_NAME_x,LINE_NAME_y,BOROUGH,WEEKDAY,DESC,GTFS_LATITUDE,GTFS_LONGITUDE,LINE,REMOTE,BOOTH,SCP,ID,ZIPCODE
11662,318,34 ST-PENN STA,IRT,2021-05-12,6626,34 ST-PENN STA,34 St - Penn Station,123,123,M,1,REGULAR,40.750373,-73.991057,Broadway - 7Av,R031,R141,00-00-00,R141 - R031 - 00-00-00,10001
1291,129,HALSEY ST,BMT,2021-05-11,2924,HALSEY ST,Halsey St,L,L,Q,1,REGULAR,40.695602,-73.904084,Canarsie,R266,H028,00-00-00,H028 - R266 - 00-00-00,11385
6416,222,ROOSEVELT IS,IND,2021-05-12,2585,ROOSEVELT ISLND,Roosevelt Island,F,F,M,1,REGULAR,40.759145,-73.953260,63rd St,R259,N602,00-00-00,N602 - R259 - 00-00-00,10044
18464,436,148 ST-LENOX,IRT,2021-06-13,769,HARLEM 148 ST,Harlem - 148 St,3,3,M,0,REGULAR,40.823880,-73.936470,Lenox - White Plains Rd,R345,R309,00-00-00,R309 - R345 - 00-00-00,10039
16337,395,103 ST,IRT,2021-06-21,5233,103 ST,103 St,6,6,M,1,REGULAR,40.790600,-73.947478,Lexington Av,R180,R252,00-00-00,R252 - R180 - 00-00-00,10037
24229,623,CANAL ST,BMT,2021-05-14,36759,CANAL ST,Canal St,JNQRZ6,JNQRZ6W,M,1,REGULAR,40.719527,-74.001775,Broadway,R118,A066,00-00-00,A066 - R118 - 00-00-00,10003
5717,210,NORWOOD-205 ST,IND,2021-06-14,3572,NORWOOD 205 ST,Norwood - 205 St,D,D,Bx,1,REGULAR,40.874811,-73.878855,Concourse,R157,N224,00-00-00,N224 - R157 - 00-00-00,10467
21097,54,SHEEPSHEAD BAY,BMT,2021-06-23,5878,SHEEPSHEAD BAY,Sheepshead Bay,BQ,BQ,Bk,1,REGULAR,40.586896,-73.954155,Broadway - Brighton,R136,B027,00-00-00,B027 - R136 - 00-00-00,11235
18337,433,JACKSON AVE,IRT,2021-06-08,1758,JACKSON AV,Jackson Av,25,25,Bx,1,REGULAR,40.816490,-73.907807,Lenox - White Plains Rd,R405,R312,00-00-00,R312 - R405 - 00-00-00,10459
20693,49,AVE J,BMT,2021-05-16,1572,AVENUE J,Avenue J,BQ,BQ,Bk,0,REGULAR,40.625039,-73.960803,Broadway - Brighton,R228,B021,00-00-00,B021 - R228 - 00-00-00,11234


#### Double check for NaNs, then save daily_per_station_2021 to file (can reload below and avoid running all of above in future)

In [82]:
daily_per_station_2021[daily_per_station_2021.isna().any(axis=1)].shape[0]

0

In [83]:
# Okay, write this dataframe to file and then we don't have to run any of the above again!

daily_per_station_2021.to_csv('daily_per_station_2021.csv', index=False)

## Move to Project_Notebook_Visualizations & start by loading in the 2 datasets just saved!